Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [95]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
import math

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [33]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():  
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    beta = tf.placeholder(tf.float32)
    
    weights = tf.Variable(
      tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))

    logits = tf.matmul(tf_train_dataset, weights) + biases
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + beta * tf.nn.l2_loss(weights)

    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
      tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)


In [36]:
num_steps = 801
feed_dict = { beta: 1e-2}
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
                predictions, train_labels[:train_subset, :]))

            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 47.026237
Training accuracy: 10.3%
Validation accuracy: 13.1%
Loss at step 100: 11.866411
Training accuracy: 73.5%
Validation accuracy: 72.4%
Loss at step 200: 4.489268
Training accuracy: 79.0%
Validation accuracy: 76.9%
Loss at step 300: 1.985249
Training accuracy: 82.6%
Validation accuracy: 79.8%
Loss at step 400: 1.135564
Training accuracy: 84.0%
Validation accuracy: 81.3%
Loss at step 500: 0.843745
Training accuracy: 84.5%
Validation accuracy: 82.0%
Loss at step 600: 0.742281
Training accuracy: 84.5%
Validation accuracy: 82.1%
Loss at step 700: 0.706677
Training accuracy: 84.5%
Validation accuracy: 82.2%
Loss at step 800: 0.694094
Training accuracy: 84.5%
Validation accuracy: 82.3%
Test accuracy: 89.0%


In [73]:
batch_size = 128
hidden_layer_size = 1024
graph = tf.Graph()
with graph.as_default():

    tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    beta = tf.placeholder(tf.float32)
    
      # Variables.
    w1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    b1 = tf.Variable(tf.zeros([hidden_layer_size]))

    w2 = tf.Variable(
        tf.truncated_normal([hidden_layer_size, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)
    logits = tf.matmul(hidden_layer, w2) + b2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + \
        beta * (tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    valid_hidden_layer = tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden_layer, w2) + b2)
    
    test_hidden_layer = tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1)
    test_prediction = tf.nn.softmax(tf.matmul(test_hidden_layer, w2) + b2)

In [74]:
num_steps = 3001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
      # Pick an offset within the training data, which has been randomized.
      # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : 1e-3}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
              valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 733.492859
Minibatch accuracy: 7.8%
Validation accuracy: 31.3%
Minibatch loss at step 500: 195.157089
Minibatch accuracy: 80.5%
Validation accuracy: 80.8%
Minibatch loss at step 1000: 115.050644
Minibatch accuracy: 84.4%
Validation accuracy: 81.5%
Minibatch loss at step 1500: 68.946182
Minibatch accuracy: 78.1%
Validation accuracy: 81.2%
Minibatch loss at step 2000: 41.516415
Minibatch accuracy: 81.2%
Validation accuracy: 84.3%
Minibatch loss at step 2500: 24.970385
Minibatch accuracy: 93.8%
Validation accuracy: 86.0%
Minibatch loss at step 3000: 15.558923
Minibatch accuracy: 85.2%
Validation accuracy: 86.6%
Test accuracy: 93.1%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [75]:
graph = tf.Graph()
with graph.as_default():

    tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
      # Variables.
    w1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    b1 = tf.Variable(tf.zeros([hidden_layer_size]))

    w2 = tf.Variable(
        tf.truncated_normal([hidden_layer_size, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)
    logits = tf.matmul(hidden_layer, w2) + b2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    valid_hidden_layer = tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden_layer, w2) + b2)
    
    test_hidden_layer = tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1)
    test_prediction = tf.nn.softmax(tf.matmul(test_hidden_layer, w2) + b2)

In [76]:
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        step_modified = np.random.randint(2)
        offset = (step_modified * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch with offset between 1 and 5, limiting the batches to those from 1 to 5.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
              valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 292.650696
Minibatch accuracy: 6.2%
Validation accuracy: 29.7%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Minibatch loss at step 1500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Minibatch loss at step 3000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 73.8%
Test accuracy: 81.0%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [77]:
graph = tf.Graph()
with graph.as_default():

    tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
      # Variables.
    w1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    b1 = tf.Variable(tf.zeros([hidden_layer_size]))

    w2 = tf.Variable(
        tf.truncated_normal([hidden_layer_size, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))

    # Training computation.
    hidden_layer = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)
    hidden_layer_drop = tf.nn.dropout(hidden_layer, 0.5)
    logits = tf.matmul(hidden_layer_drop, w2) + b2
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    valid_hidden_layer = tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_hidden_layer, w2) + b2)
    
    test_hidden_layer = tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1)
    test_prediction = tf.nn.softmax(tf.matmul(test_hidden_layer, w2) + b2)

In [78]:
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        step_modified = np.random.randint(2)
        offset = (step_modified * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch with offset between 1 and 5, limiting the batches to those from 1 to 5.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
              valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 464.351654
Minibatch accuracy: 10.2%
Validation accuracy: 27.9%
Minibatch loss at step 500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 75.8%
Minibatch loss at step 1000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 75.7%
Minibatch loss at step 1500: 0.087595
Minibatch accuracy: 99.2%
Validation accuracy: 76.0%
Minibatch loss at step 2000: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 75.6%
Minibatch loss at step 2500: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 76.0%
Minibatch loss at step 3000: 0.581117
Minibatch accuracy: 99.2%
Validation accuracy: 75.7%
Test accuracy: 83.1%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [125]:
graph = tf.Graph()
with graph.as_default():

    tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    beta = tf.placeholder(tf.float32)
    
      # Variables.
    w1 = tf.Variable(
        tf.truncated_normal([image_size * image_size, hidden_layer_size]))
    b1 = tf.Variable(tf.zeros([hidden_layer_size]))

    w2 = tf.Variable(
        tf.truncated_normal([hidden_layer_size, hidden_layer_size], stddev=math.sqrt(2.0/(hidden_layer_size))))
    b2 = tf.Variable(tf.zeros([hidden_layer_size]))
    
    w3 = tf.Variable(
        tf.truncated_normal([hidden_layer_size, hidden_layer_size], stddev=math.sqrt(2.0/(hidden_layer_size))))
    b3 = tf.Variable(tf.zeros([hidden_layer_size]))
    
    w4 = tf.Variable(
        tf.truncated_normal([hidden_layer_size, hidden_layer_size], stddev=math.sqrt(2.0/(hidden_layer_size))))
    b4 = tf.Variable(tf.zeros([hidden_layer_size]))

    w5 = tf.Variable(
        tf.truncated_normal([hidden_layer_size, num_labels], stddev=math.sqrt(2.0/(hidden_layer_size))))
    b5 = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    h1 = tf.nn.relu(tf.matmul(tf_train_dataset, w1) + b1)
    h1_drop = tf.nn.dropout(h1, 1)
    
    h2 = tf.nn.relu(tf.matmul(h1_drop, w2) + b2)
    h2_drop = tf.nn.dropout(h2, 1)
    
    h3 = tf.nn.relu(tf.matmul(h2_drop, w3) + b3)
    h3_drop = tf.nn.dropout(h3, 1)
    
    h4 = tf.nn.relu(tf.matmul(h3_drop, w4) + b4)
    h4_drop = tf.nn.dropout(h4, 1)
    
    logits = tf.matmul(h4, w5) + b5
    
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + \
        beta * (tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2) + tf.nn.l2_loss(w3) + tf.nn.l2_loss(w4) + tf.nn.l2_loss(w5))

    # Optimizer.
    optimizer = tf.train.AdamOptimizer(0.005).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    
    valid_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, w1) + b1)
    valid_2 = tf.nn.relu(tf.matmul(valid_1, w2) + b2)
    valid_3 = tf.nn.relu(tf.matmul(valid_2, w3) + b3)
    valid_4 = tf.nn.relu(tf.matmul(valid_3, w4) + b4)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_4, w5) + b5)
    
    test_1 = tf.nn.relu(tf.matmul(tf_test_dataset, w1) + b1)
    test_2 = tf.nn.relu(tf.matmul(test_1, w2) + b2)
    test_3 = tf.nn.relu(tf.matmul(test_2, w3) + b3)
    test_4 = tf.nn.relu(tf.matmul(test_3, w4) + b4)
    test_prediction = tf.nn.softmax(tf.matmul(test_4, w5) + b5)

In [126]:
num_steps = 10001
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
      # Pick an offset within the training data, which has been randomized.
      # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset + batch_size), :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, beta : 1e-4}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
              valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 39.005882
Minibatch accuracy: 7.8%
Validation accuracy: 12.5%
Minibatch loss at step 500: 11.831876
Minibatch accuracy: 86.7%
Validation accuracy: 82.4%
Minibatch loss at step 1000: 3.531437
Minibatch accuracy: 89.1%
Validation accuracy: 84.7%
Minibatch loss at step 1500: 1.466405
Minibatch accuracy: 84.4%
Validation accuracy: 84.0%
Minibatch loss at step 2000: 0.922169
Minibatch accuracy: 82.0%
Validation accuracy: 84.7%
Minibatch loss at step 2500: 0.615229
Minibatch accuracy: 87.5%
Validation accuracy: 84.9%
Minibatch loss at step 3000: 0.725984
Minibatch accuracy: 82.8%
Validation accuracy: 85.8%
Minibatch loss at step 3500: 0.690094
Minibatch accuracy: 82.0%
Validation accuracy: 85.0%
Minibatch loss at step 4000: 0.542142
Minibatch accuracy: 85.9%
Validation accuracy: 85.7%
Minibatch loss at step 4500: 0.655435
Minibatch accuracy: 83.6%
Validation accuracy: 85.9%
Minibatch loss at step 5000: 0.517604
Minibatch accuracy: 87.5%
Validation accura